In [2]:
!git clone https://github.com/piEsposito/br-quad-2.0

Cloning into 'br-quad-2.0'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 27 (delta 8), reused 20 (delta 4), pack-reused 0
Receiving objects: 100% (27/27), 11.56 MiB | 8.96 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [65]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00


In [13]:
from html.entities import name2codepoint
import re

REGEX_PATTERN = re.compile(r'(?<=\&)\w+(?=\;)')

def replace_htmlcodecs(text):

    for match_obj in REGEX_PATTERN.finditer(text):

        str_pattern = match_obj.group()
        text = re.sub(f'\&{str_pattern}\;', chr(name2codepoint[str_pattern]), text)

    return text


In [62]:


count = 0
fail = 0

def preprocess_data(data_collection):
    examples = []
    for example in data:

      title = example['title']
      paragraphs = example["paragraphs"]

      for paragraph in paragraphs:
        possible_qas = [qa for qa in paragraph["qas"] if not qa["is_impossible"]]
        context = paragraph['context']

        for qa in possible_qas:

          for answer in qa["answers"]:

              answer_start = answer["answer_start"]
              answer_text = replace_htmlcodecs(answer["text"])
              answer_len = len(answer_text)

              if answer_start != 0 and (paragraph['context'][answer_start:answer_start+answer_len] == answer_text):
                  answer_ctx = list(context)
                  answer_ctx.insert(answer_start, " <h1> ")
                  answer_ctx.insert(answer_start + answer_len + 1, " <h1> ")
                  answer_ctx =  "".join(answer_ctx)
                  question = qa["question"]
                  examples.append({
                        "paragraph": replace_htmlcodecs(paragraph["context"]),
                        "paragraph_id": paragraph["qas"][0]["id"],
                        "question": question,
                        "answer": answer_text,
                        "paragraph_answer": replace_htmlcodecs(answer_ctx),
                        "paragraph_question": f"question: {question} paragraph: {context}",
                    })
                  break
              else:
                continue
    return examples

In [63]:
import json

with open('/content/br-quad-2.0/data/brquad-gte-train-v2.0.json') as f:
  data = json.load(f)["data"]
  train_examples = preprocess_data(data)

with open('/content/br-quad-2.0/data/brquad-gte-dev-v2.0.json') as f:
  data = json.load(f)["data"]
  dev_examples = preprocess_data(data)

In [67]:
import pandas as pd
import datasets as dts
from datasets import DatasetDict

dataset = DatasetDict({
    "vbralidation": dts.Dataset.from_pandas(pd.DataFrame(dev_examples)),
    "train": dts.Dataset.from_pandas(pd.DataFrame(train_examples))
})
dataset

DatasetDict({
    validation: Dataset({
        features: ['paragraph', 'paragraph_id', 'question', 'answer', 'paragraph_answer', 'paragraph_question'],
        num_rows: 1946
    })
    train: Dataset({
        features: ['paragraph', 'paragraph_id', 'question', 'answer', 'paragraph_answer', 'paragraph_question'],
        num_rows: 55135
    })
})

In [68]:
HF_TOKEN = ""
dataset.push_to_hub("tiagoblima/br_squadv2", token=HF_TOKEN)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/56 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/tiagoblima/br_squadv2/commit/72ab0b099cb1b5006f7d08a8809e24163bfc77de', commit_message='Upload dataset', commit_description='', oid='72ab0b099cb1b5006f7d08a8809e24163bfc77de', pr_url=None, pr_revision=None, pr_num=None)